# MODS203 Project
## Ryan Borhani, Mathilde Froger, Apolline Isaia, Solal Urien
### Load libraries

In [27]:
import pandas as pd
import urllib.request
import re
import requests
import random
import unidecode

from bs4 import BeautifulSoup as bs
from urllib.request import Request, urlopen

### List of towns

In [33]:
my_file = open("Communes.txt", "r")
data = my_file.read()
town_list = data.split("\n")
my_file.close()

### Normalizing the names

In [34]:
for k in range(len(town_list)) :
    town_list[k] = unidecode.unidecode(town_list[k].lower())
    town_list[k] = town_list[k].replace('\'','-')
    town_list[k] = town_list[k].replace(' ','-')
    town_list[k] = town_list[k].replace('ç','c')
len(town_list)

190

### Adding the *arrondissements* of Paris, Marseille and Lyon

In [35]:
for k in range (1,21): #Paris has 20 arrondissements
    town_list.insert(1,'paris-'+str(75000+k))
    if k<17: #Marseille has 16 arrondissements
        town_list.insert(2+k,'marseille-'+str(13000+k))
    if k<10: #Lyon has 9 arrondissements
        town_list.insert(3+2*k,'lyon-'+str(69000+k))

In [37]:
town_list

['paris',
 'paris-75020',
 'paris-75019',
 'paris-75018',
 'paris-75017',
 'paris-75016',
 'paris-75015',
 'paris-75014',
 'paris-75013',
 'paris-75012',
 'paris-75011',
 'paris-75010',
 'paris-75009',
 'paris-75008',
 'paris-75007',
 'paris-75006',
 'paris-75005',
 'paris-75004',
 'paris-75003',
 'paris-75002',
 'paris-75001',
 'marseille',
 'marseille-13016',
 'marseille-13015',
 'marseille-13014',
 'marseille-13013',
 'marseille-13012',
 'marseille-13011',
 'marseille-13010',
 'marseille-13009',
 'marseille-13008',
 'marseille-13007',
 'marseille-13006',
 'marseille-13005',
 'marseille-13004',
 'marseille-13003',
 'marseille-13002',
 'marseille-13001',
 'lyon',
 'lyon-69009',
 'lyon-69008',
 'lyon-69007',
 'lyon-69006',
 'lyon-69005',
 'lyon-69004',
 'lyon-69003',
 'lyon-69002',
 'lyon-69001',
 'toulouse',
 'nice',
 'nantes',
 'montpellier',
 'strasbourg',
 'bordeaux',
 'lille',
 'rennes',
 'reims',
 'saint-etienne',
 'le-havre',
 'toulon',
 'grenoble',
 'dijon',
 'angers',
 'nimes'

### Make the request and extract the HTML code
For all departments, we request the HTML code. For this doing, we use the previous list and convert the strings into the model that is used for the *Doctolib* URL adress.
The collection of the data was particularly complex given the reaction of the *Doctolib* website.

**Warning 1: the repeated requests might provoke a 403 HTTP error, using a VPN might therefore be useful**

**Warning 2: The code might be particularly long to compute** 

In [40]:
hdr = {'User-Agent': 'Mozilla/6.0'}
list_HTML = []

for town in town_list:
    url = 'https://www.doctolib.fr/dentiste/'+town
    req = Request(url,headers=hdr)
    page = urlopen(req)
    soup = bs(page,'html.parser')
    list_HTML.append(soup)

In [41]:
len(list_HTML)

235

This list has to be heavily processed before we can add the information to a Dataframe.

## Data Processing
We previously learned (1st part of the project) how the HTML code was structured.
We therefore divide the list in different parts, each containing the information of all the doctors in a given department.

In [42]:
all_doc = []
for dep_doc in list_HTML:
    all_doc = all_doc + [str(dep_doc.findAll(type= "application/ld+json")[1])]
len(all_doc)

235

In [43]:
all_doc[0]

'<script type="application/ld+json">[{"@context":"http://schema.org/","@type":"Physician","name":"Céline BISMUTH-SLAMA","medicalSpecialty":"Chirurgien-dentiste","legalName":"","url":"/dentiste/paris/celine-bismuth-slama","address":{"@type":"PostalAddress","name":"Dentylis - Turbigo Châtelet","streetAddress":"7 Rue de Turbigo","postalCode":"75001","addressLocality":"Paris"},"paymentAccepted":""},{"@context":"http://schema.org/","@type":"Physician","name":"Yves GOLDFARB","medicalSpecialty":"Chirurgien-dentiste","legalName":null,"url":"/dentiste/paris/yves-goldfarb","address":{"@type":"PostalAddress","name":"Cabinet dentaire ","streetAddress":"17 Rue la Fayette","postalCode":"75009","addressLocality":"Paris"},"paymentAccepted":"Cash, Check, Credit card"},{"@context":"http://schema.org/","@type":"Physician","name":"Jorge SIMONETTI","medicalSpecialty":"Chirurgien-dentiste","legalName":"","url":"/dentiste/paris/jorge-simonetti","address":{"@type":"PostalAddress","name":"","streetAddress":"28

We subsequently divide the list, each part containing all the information of a single doctor.

In [44]:
list_inf = []

for depart_doc in all_doc:
    cur_nb = depart_doc.find('{') #our indice
    cur_parent = 1 #nb of parenthesis to close

    for k in range (cur_nb+1, len(depart_doc)):
        if (depart_doc[k]=='{'):
            cur_parent +=1
        elif (depart_doc[k] == '}'):
            cur_parent -=1 
            if (cur_parent == 0): #we found the parenthesis that closes the first parenthesis
                list_inf.append(depart_doc[cur_nb+1:k])
                cur_nb = depart_doc.find('{',k)
len(list_inf)

5128

We now have numerous strings, each describing a doctor.
We are going to process these strings one by one to obtain exactly the dataframe we want. We also add more information to the DataFrame with the *Payment_accepted* column.

In [45]:
list_name, list_type, list_spe,list_adr,list_post_cod, list_cit, list_pay = [],[],[],[],[],[],[]

for doc in list_inf: #boucle pour les noms des médecins ou des hopitals
    i=0
    cur_nb = doc.find('name')
    while (doc[cur_nb+6+i] != ','):
        i+=1
    list_name.append(doc[cur_nb+7:cur_nb+5+i])

for doc in list_inf: #boucle pour les type des médecins/hopitals
    i=0
    cur_nb = doc.find('type')
    while (doc[cur_nb+6+i] != ','):
        i+=1
    list_type.append(doc[cur_nb+7:cur_nb+5+i])

for doc in list_inf: #boucle pour les spécialités
    i=0
    cur_nb = doc.find('medicalSpecialty')
    while (doc[cur_nb+18+i] != ','):
        i+=1
    list_spe.append(doc[cur_nb+19:cur_nb+17+i])

for doc in list_inf: #boucle pour les villes
    i=0
    cur_nb = doc.find('addressLocality')
    while (doc[cur_nb+16+i] != ','):
        i+=1
    list_cit.append(doc[cur_nb+18:cur_nb+14+i])

for doc in list_inf: #boucle pour les adresses (rue et numéro)
    i=0
    cur_nb = doc.find('streetAddress')
    while (doc[cur_nb+6+i] != ','):
        i+=1
    list_adr.append(doc[cur_nb+16:cur_nb+5+i])

for doc in list_inf:#boucle pour les codes postaux
    i=0
    cur_nb = doc.find('postalCode')
    while (doc[cur_nb+11+i] != ','):
        i+=1
    try:
        list_post_cod.append(int(doc[cur_nb+13:cur_nb+10+i].replace(' ',''))) # le code postal est un entier, on applique donc un rapide formatage avant de transformer le string en int
    except:
        list_post_cod.append(doc[cur_nb+13:cur_nb+10+i]) #Si la transformation en integer n'est pas possible, on laisse le format string

for doc in list_inf: #boucle pour les moyens de paiement
    i=0
    cur_nb = doc.find('paymentAccepted')
    while (cur_nb+i != len(doc)):
        i+=1
    list_pay.append(doc[cur_nb+18:len(doc)-1])

data = {'Name':list_name,'Type':list_type, 'Speciality':list_spe,'Address':list_adr,'City':list_cit,'Postal_Code':list_post_cod, 'Payment_accepted':list_pay}
df = pd.DataFrame(data)
display(df)

,Name,Type,Speciality,Address,City,Postal_Code,Payment_accepted
0,Céline BISMUTH-SLAMA,Physician,Chirurgien-dentiste,7 Rue de Turbigo,Paris,75001,
1,Yves GOLDFARB,Physician,Chirurgien-dentiste,17 Rue la Fayette,Paris,75009,"Cash, Check, Credit card"
2,Jorge SIMONETTI,Physician,Chirurgien-dentiste,28 Rue de Ponthieu,Paris,75008,"Cash, Check, Credit card"
3,Dentiste Consultation/rdv/urgence,Physician,Chirurgien-dentiste,116 Boulevard Pereire,Paris,75017,
4,Patrick BERMOT,Physician,Chirurgien-dentiste,11 Rue Magellan,Paris,75008,"Cash, Check, Credit card"
...,...,...,...,...,...,...,...
5123,Xuan-Lan LE VAN CUU,Physician,Chirurgien-dentiste,21 Avenue Aristide Briand,Cachan,94230,"Cash, Check, Credit card"
5124,Catherine CARAMIGEAS,Physician,Chirurgien-dentiste,21 Rue de l'Avenir,Fontenay-aux-Roses,92260,"Cash, Check, Credit card"
5125,Estelle AZIZA,Physician,Chirurgien-dentiste,29-31 Avenue du Général Leclerc,Bourg-la-Reine,92340,"Cash, Check, Credit card"
5126,Quoc Hung TRAN,Physician,Chirurgien-dentiste,29-31 Avenue du Général Leclerc,Bourg-la-Reine,92340,"Cash, Check, Credit card"


### Export the Data

In [47]:
df.to_csv(r"Dataframe-Dentist.csv", index=False)